In [3]:
# %pip install boto3==1.34.93 ipython --quiet

%load_ext autoreload
%autoreload 2

# PIPELINE

## STRAT 1 BIS

In [4]:
version = "v1"
dossier = "2023DOS0550696"

# AO
cdc_json_file_path = f"/workspace/docs/source/notebooks/json/{dossier}/{version}/ao/formatted_json_cdc.json"
rec_json_file_path = f"/workspace/docs/source/notebooks/json/{dossier}/{version}/ao/formatted_json_rec.json"
# CV
entreprise = "cgi"
cv_json_file_path = f"/workspace/docs/source/notebooks/json/{dossier}/{version}/{entreprise}/formatted_json_cv.json"

In [5]:
import json 

# Open the JSON file and load its contents
with open(cdc_json_file_path, "r") as file:
    cdc_json_data = json.load(file)
with open(cv_json_file_path, "r") as file:
    cv_json_data = json.load(file)
with open(rec_json_file_path, "r") as file:
    rec_json_data = json.load(file)

# Now you can work with the loaded JSON data
print(cdc_json_data)
print(cv_json_data)

# Add you json inputs here
REQUIREMENTS = cdc_json_data
RESUME = cv_json_data

{'objectif_de_la_mission': ["La mission objet de cette consultation vise à décharger les équipes SNCF dans trois domaines concernant les services bureautiques et digitaux au sens large : - La gestion courante des sollicitations et demandes ; - L'assistance et le conseil aux utilisateurs ; - La gestion des stocks, de l'approvisionnement à la mise au rebus en passant par la remise en main.", 1], 'duree_de_la_mission': [210, 3], 'nombre_de_profils': [1, 2], 'types_de_profils': [{'type': ['701.E.PAR - Support Méthode, qualité et Sécurité Expert - ouvert à des profils Senior (701.S.PAR)', 2]}], 'taches_et_responsabilites': [{'tache_et_responsabilite': ['Traiter les demandes des agents G&C reçues via le portail interne Digital Office', 1]}, {'tache_et_responsabilite': ["Prendre en charge les déclarations d'incidents ou d'anomalie de fonctionnement bureautique quand les processus standard des porteurs de services ne suffisent pas", 1]}, {'tache_et_responsabilite': ['Alerter / escalader en int

In [6]:
CRITERIAS  = rec_json_data['sous_criteres_techniques_et_ponderation']

CRITERIAS

[{'critere': ['Qualité de la réponse (Compréhension du besoin, le cas échéant outils et méthodologie proposés)',
   1],
  'ponderation': [10, 1]},
 {'critere': ['Nombre et pertinence des expériences similaires', 1],
  'ponderation': [30, 1]},
 {'critere': ['Compétences techniques', 1], 'ponderation': [30, 1]},
 {'critere': ['Connaissance du domaine fonctionnel', 1],
  'ponderation': [20, 1]},
 {'critere': ["Expériences similaires en accompagnement au changement, actions pédagogiques illustrant la capacité d'écoute, de reformulation et de synthèse",
   1],
  'ponderation': [10, 1]}]

In [7]:
SCALE = {
    0: "Ne répond pas au besoin",
    1: "Ne répond pas suffisamment au besoin",
    2: "Répond partiellement au besoin",
    3: "Répond correctement au besoin",
    4: "Très bonne réponse",
    5: "Réponse au-delà des attentes"
}

### 1 appel LLM pour tous les criteres

In [7]:
from dag_nao.modules.technical_notation.compare_experiences.llm_comparator import LLMComparator

In [8]:
comp = LLMComparator()

In [9]:
a, b, c = comp.call_comparator(REQUIREMENTS, RESUME, CRITERIAS, SCALE)

Note technique : 3.5


In [11]:
print(a)

Voici mon analyse détaillée du CV par rapport aux critères fournis :

<scratchpad>
Critère 1 : Qualité de la réponse (Compréhension du besoin, le cas échéant outils et méthodologie proposés)
- Le CV ne mentionne pas explicitement la compréhension du besoin ou les outils/méthodologies proposés pour cette mission spécifique. Il décrit les expériences et compétences générales du candidat.
=> Score : 2 (Répond partiellement au besoin)

Critère 2 : Nombre et pertinence des expériences similaires
- Le candidat a une longue expérience de plus de 20 ans dans des rôles similaires de support technique, gestion des utilisateurs, administration de parc informatique, etc.
- Ses expériences chez Avancial, SFR et SNCF semblent très pertinentes pour les tâches décrites dans l'appel d'offre.
=> Score : 4 (Très bonne réponse)

Critère 3 : Compétences techniques  
- Le CV montre une solide expertise technique dans les domaines requis : gestion de flottes mobiles Android/iOS, parc PC Windows/Mac, environn

In [13]:
b

{'Qualité de la réponse (Compréhension du besoin, le cas échéant outils et méthodologie proposés)': {'justification': 'Le CV ne mentionne pas explicitement la compréhension du besoin ou les outils/méthodologies proposés pour cette mission spécifique. Il décrit les expériences et compétences générales du candidat.',
  'score': 2},
 'Nombre et pertinence des expériences similaires': {'justification': "Le candidat a une longue expérience de plus de 20 ans dans des rôles similaires de support technique, gestion des utilisateurs, administration de parc informatique, etc. Ses expériences chez Avancial, SFR et SNCF semblent très pertinentes pour les tâches décrites dans l'appel d'offre.",
  'score': 4},
 'Compétences techniques': {'justification': 'Le CV montre une solide expertise technique dans les domaines requis : gestion de flottes mobiles Android/iOS, parc PC Windows/Mac, environnement O365/Active Directory/VPN. Le candidat a suivi de nombreuses formations techniques pertinentes.',
  's

In [14]:
print(c)

3.5


### 1 appel LLM par critere

In [8]:
from dag_nao.modules.technical_notation.compare_experiences_per_criteria.llm_comparator import LLMComparator

In [9]:
comp2 = LLMComparator()

In [10]:
a, b, c = comp2.call_comparator(REQUIREMENTS, RESUME, CRITERIAS, SCALE)

Note technique : 3.0999999999999996


In [13]:
b

{'Qualité de la réponse (Compréhension du besoin, le cas échéant outils et méthodologie proposés)': {'justification': "Le candidat a une expérience pertinente en support bureautique, gestion de parc et qualité, avec des tâches similaires à celles décrites dans l'appel d'offre. Cependant, son CV ne détaille pas suffisamment sa compréhension spécifique des besoins liés à l'environnement O365/Active Directory/VPN demandé, ni les outils et méthodologies qu'il pourrait proposer. Certaines expériences semblent également un peu anciennes. Dans l'ensemble, le CV répond partiellement au besoin sur ce critère.",
  'score': 2},
 'Nombre et pertinence des expériences similaires': {'justification': "Le candidat possède 6 ans d'expérience très pertinente en tant que Support Méthode, qualité et Sécurité chez AVANCIAL, où il gérait les incidents, le support bureautique et la documentation, ce qui correspond aux principales responsabilités de la mission. Il a également 6 ans d'expérience en tant que Re

In [14]:
c

3.0999999999999996